In [2]:
import os
import numpy as np
import torch
from PIL import Image
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

import gc
import numpy as np
import pandas as pd
import torch
import os
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from torchvision.datasets import DatasetFolder, VisionDataset
# This is for the progress bar.
from tqdm.auto import tqdm
import random
from pathlib import Path
import math
import cv2


In [3]:
def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights="DEFAULT")
    # return model
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)
    
    return model

In [4]:
import transforms as T

def get_transform(train):
    transforms = []
    transforms.append(T.PILToTensor())
    # transforms.append(T.ConvertImageDtype(torch.float))
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [5]:
def get_valid_data(labels):
    valid_data = []
    for i in range(len(labels)):
        for j in range(len(labels.iloc[i]['annotations'])):
            if labels.iloc[i]['annotations'][j]['type'] == "blood_vessel":
                valid_data.append(labels.iloc[i]['id'])
                break
    
    return valid_data

def gen_mask(annotations):
    boxes = []
    masks = []
    # area = []
    for i in range(len(annotations)):

        if annotations[i]['type'] == "blood_vessel":
            pts = np.array(annotations[i]['coordinates'])
            # print(pts[0][0])
            min_xy = np.min(pts, axis=1)[0]
            max_xy = np.max(pts, axis=1)[0]

            boxes += [np.concatenate((min_xy, max_xy), axis=0)]

            mask = np.zeros((512,512), dtype=np.uint8)
            cv2.fillPoly(mask, pts, 1)

            # area.append((max_xy[0] - min_xy[0]) * (max_xy[1] - min_xy[1]))

            masks += [mask]
        # break
    boxes = torch.as_tensor(boxes, dtype=torch.float32)
    masks = torch.as_tensor(masks, dtype=torch.uint8)
    area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])

    return boxes , masks , area

In [7]:
class KidneyDataset(Dataset):
    def __init__(self , labels , metadata , image_list , tfm):
        self.labels = labels
        self.matadata = metadata
        self.image_list = image_list
        self.tfm = tfm
        
        self.image_with_target = {}

        image_path = Path(image_list[0]).parent
        print(image_path)
        
        for idx , col in labels.iterrows():
            


            # if f"{image_path}\\{col['id']}.tif" in image_list:
            if f"{image_path}/{col['id']}.tif" in image_list:
                polygons = col['annotations']
                # print(polygons[0])
                boxes , masks , area = gen_mask(polygons)
                iscrowd = torch.zeros((len(boxes),), dtype=torch.int64)
                
                target = {}
                target["boxes"] = boxes
                target["labels"] = torch.zeros((len(boxes),), dtype=torch.int64)
                target["masks"] = masks
                target["image_id"] = torch.tensor([idx])
                target["area"] = area
                target["iscrowd"] = iscrowd
                self.image_with_target[col['id']] = target

        # print()

    def __len__(self):
        return len(self.image_list)
    
    def __getitem__(self,idx):
        
        # id = self.image_list[idx].split('\\')[-1].split('.')[0]
        id = self.image_list[idx].split('/')[-1].split('.')[0]
        img = Image.open(self.image_list[idx])
        target = self.image_with_target[id]


        if self.tfm is not None:
            img, target = self.tfm(img, target)
        # print(target)
        return img , target
        

In [8]:
ROOT = Path("hubmap-hacking-the-human-vasculature")
polygon_file = "polygons.jsonl"
image_folder = "train"
metadata_file = "tile_meta.csv"
train_valid_ratio = 0.9

labels = pd.read_json(ROOT / polygon_file , lines=True)
id_list = get_valid_data(labels)
metadata = pd.read_csv(ROOT / metadata_file)
image_list = [str(i) for i  in (ROOT / image_folder).glob('*.tif') if i.stem in id_list]
train_list = image_list[:int(len(image_list)*train_valid_ratio)]
valid_list = image_list[int(len(image_list)*train_valid_ratio):]

In [9]:
train_set = KidneyDataset(labels , metadata , valid_list , get_transform(train=True))

# img.shape
# target['masks'].shape
# target["labels"]
# target["boxes"].shape
# pass

hubmap-hacking-the-human-vasculature\train


C:\Users\arthu\AppData\Local\Temp\ipykernel_16760\2335065411.py:32: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:248.)
  boxes = torch.as_tensor(boxes, dtype=torch.float32)


In [13]:
img , target =  train_set[0]
target
print(len(train_set))

163


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize a model, and put it on the device specified.
model = get_model_instance_segmentation(1).to(device)
batch_size = 2
n_epochs = 20
patience = 10
optimizer = torch.optim.SGD(model.parameters(), lr=0.0003, weight_decay=1e-5)
_exp_name = "mask-rcnn-model"
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer , step_size=3 , gamma=0.1)

In [ ]:
import utils
train_set = KidneyDataset(Path("hubmap-hacking-the-human-vasculature") , "polygons.jsonl" , "tile_meta.csv" , "train" , get_transform(train=True))
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True , collate_fn=utils.collate_fn)
# valid_set = KidneyDataset(Path("hubmap-hacking-the-human-vasculature") , "polygons.jsonl" , "tile_meta.csv" , "train" , get_transform(train=False))
# valid_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
# trainset[0]


In [ ]:
num_epochs = 10
from engine import train_one_epoch, evaluate

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, train_loader , device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()

In [ ]:
# img = Image.open("test.jpg").convert("RGB")
# # img = tfm(img)
# model = get_model_instance_segmentation(2)
# model.eval()
 
# pred = model(img.unsqueeze(0))
# print(pred)

In [ ]:
# print(pred[0]['masks'].shape)
# Image.fromarray(pred[0]['masks'][0,0].mul(255).byte().cpu().numpy())